In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
Phase_1 = False
Phase_2 = False
Phase_3 = False
Phase_4 =  True

DATA_FILE = "sc2021_train_deals.csv"
TARGET = "target"

# Range of values for selection in phase 1 :
import numpy as np
N_ESTIMATORS_RANG = list(range(10,500,10))
RANG_LEARNING_RATE =  list(np.arange(0.01,0.1,0.001)) 

RSCV_N_TRIALS = 1500
N_TRIALS = 1500
SEED = 82736

# Path definition
import pathlib
DATA_DIR = pathlib.Path(".")
LOG_PATH = pathlib.Path("./log/")
NAME_RUN_LOG = 'Run_CATBOOST_Logs.log'

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
CAT_COLS = ["material_code", "company_code",  "region",  "month"]

In [3]:
#{'tr__cat__enc': TargetEncoder(),'ml__n_estimators': 1670,'ml__learning_rate': 0.006} -0.5304044235821974

params = { 
    'n_estimators': 106, #1670, #106, #316, #34, # 316 # 106	0.055	CatBoostEncoder	
    'learning_rate': 0.055, #0.006, #0.055, #0.02, #0.0813, # 0.02
    
    'max_depth': 4, 
    'num_leaves': 273, 
    'max_bin': 250, 
    'min_child_samples': 180, 
    'min_child_weight': 137.32283812349354,
    
    'bagging_fraction': 0.8276306589058549, 
    'colsample_bytree': 0.9993275026216776, 
    'subsample': 21,
    
    'reg_alpha': 1.0796993492912197, 
    'reg_lambda': 0.062264967128417226}

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
#from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error, make_scorer

import numpy as np
# импортируем класс TimeSeriesSplit 
# реализующий стратегию перекрестной проверки на временных рядах
# и класс RandomizedSearchCV для поиска гиперпараметров
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from tqdm.notebook import tqdm
from time import time
import datetime
import warnings
import dill
import math
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from category_encoders import TargetEncoder, OrdinalEncoder, LeaveOneOutEncoder, CatBoostEncoder


warnings.simplefilter(action='ignore', category=FutureWarning)
# увеличиваем количество отображаемых столбцов
pd.set_option('display.max_columns', 100)

In [5]:
start_time = time()
print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")))

Время старта скрипта: 23-12-2021 23:08


In [6]:
BASE_LOG = 4 # math.e

def exp_b_m1(x, base, r=4):
    return np.round(base**x - 1, r)

def log_b_1p(x, base):
    x.clip(lower=1.0e-200).apply(lambda y: math.log(y+1, base))
    return 

In [7]:
# загружаем набор
data = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), parse_dates=["month", "date"])

In [8]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
group_ts.head()

month                                                                2018-01-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              340.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-02-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              340.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942               20.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-03-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              260.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301               30.0   

month                                                                2018-04-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              240.0   
133           0            Белоруссия Минская обл.     10942              200.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942                0.0   
                           Казахстан  г. Нур-Султан    13301               30.0   

month                                                                2018-05-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942               60.0   
                                      Могилевская обл. 10942              140.0   
                                      г. Минск         10942               40.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-06-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942              160.0   
                           Казахстан  г. Нур-Султан    13301                0.0   

month                                                                2018-07-01  \
material_code company_code country    region           manager_code               
124           7278         Россия     Респ. Татарстан  17460              220.0   
133           0            Белоруссия Минская обл.     10942                0.0   
                                      Могилевская обл. 10942                0.0   
                                      г. Минск         10942              180.0   
          

In [9]:
# 
def get_features_Cv(df: pd.DataFrame, month: pd.Timestamp, N=1, MDAD_1=2,  MNGR_GRP_MDAD=0) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(N)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    # формируем лаги за N месяцев
    features[[f"vol_tm{i}" for i in range(N, 0, -1)]] = df.loc[:, start_period:end_period].copy()
 
    # Добавление ГРУППОВЫХ скользящих средних
    gr = "country"
    period = 10
    df2 = df.copy()
    df2[df2.columns.to_list()] = \
                           df2.groupby(level=gr).transform(lambda x: x.mean())
    grp_manager_roll_mean = df2.rolling(period, axis=1, min_periods=1)
    features = \
        features.join( grp_manager_roll_mean.mean().iloc[:, -1].rename(gr+"_grp_mean_"+str(period)))

    #  MEAN
    for period in [2,3,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.mean().iloc[:, -1].rename("avg_"+str(period)))

    #  median
    for period in [2,3,4,12]: # 10,11,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.median().iloc[:, -1].rename("median"+str(period)))
                 
    #  MAX
    for period in range(2,4,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.max().iloc[:, -1].rename("max_"+str(period)))

    features["month"] = month.month

    return features

In [10]:
#cv_traine_range = pd.date_range("2019-01-01", "2020-07-01", freq="MS")

cv_traine_range = pd.date_range("2018-07-01", "2020-07-01", freq="MS")
cv_traine_range, len(cv_traine_range)

(DatetimeIndex(['2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
                '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
                '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
                '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
                '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01',
                '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
                '2020-07-01'],
               dtype='datetime64[ns]', freq='MS'),
 25)

In [11]:
%%time

full_features = {}

tmp = group_ts.clip(lower=1.0e-200).copy()
for i in tmp.columns:
    tmp[i] = tmp[i].clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG))

dataset_features = []
for target_month in tqdm(cv_traine_range):
    features = get_features_Cv(tmp, target_month)
    features[TARGET] = group_ts[target_month]
    dataset_features.append(features.reset_index())
full_features = pd.concat(dataset_features, ignore_index=True)

#CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
CAT_COLS = ["material_code", "company_code",  "region",  "month"]
# создаем список  переменных
FTS_COLS = full_features.columns.tolist()
FTS_COLS.remove(TARGET)
#################################################################################################################################
print(FTS_COLS)

  0%|          | 0/25 [00:00<?, ?it/s]

['material_code', 'company_code', 'country', 'region', 'manager_code', 'month', 'vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']
Wall time: 18.3 s


In [12]:
FTS_COLS.remove("country")
FTS_COLS.remove("manager_code")
print(FTS_COLS)

['material_code', 'company_code', 'region', 'month', 'vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']


In [13]:
cv_features = full_features.copy()

In [14]:
# 18 групп - блоки по 1 наблюдению(в 1 наблюдении данные 941 временного ряда)
N_timeseries = 941 # в 1 наблюдении данные 941 временного ряда
start_train = 1  # 

test_size=N_timeseries #*3
#max_train_size = N_timeseries*6
n_splits= 17 #int(len(full_features2)/(test_size))-1 - start_train # 

tscv = TimeSeriesSplit(n_splits=n_splits, 
                       #max_train_size=max_train_size, 
                       test_size=test_size)

In [15]:
# 6 групп
N_timeseries = 941
start_train = 1  # 

test_size=N_timeseries *3
#max_train_size = N_timeseries*6
n_splits= 5 #int(len(full_features2)/(test_size))-1 - start_train # 

tscv = TimeSeriesSplit(n_splits=n_splits, 
                       #max_train_size=max_train_size, 
                       test_size=test_size)

In [16]:
for train_index, test_index in tscv.split(cv_features): # tscv.split(X): # tscv.split(cv_features)
    print("TRAIN: ", train_index[0],"...",train_index[-1])
    print("TEST: ", test_index[0],"...",test_index[-1])

    X_train, X_test = cv_features.iloc[train_index], cv_features.iloc[test_index]
    y_train, y_test = cv_features.iloc[train_index][TARGET], cv_features.iloc[test_index][TARGET]
    
    print('Обучающий набор: %f' % (len(X_train)/941)) #test_size))
    print('Тестовый набор: %d' % (len(X_test)/941)) #test_size))

    #display(X_train)
    display(X_test)
print("tscv.get_n_splits()=",tscv.get_n_splits())

TRAIN:  0 ... 9409
TEST:  9410 ... 12232
Обучающий набор: 10.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
9410,124,7278,Россия,Респ. Татарстан,17460,5,3.769579,1.914605,3.797553,3.806877,3.909796,3.797553,3.825526,3.825526,3.893951,3.825526,3.825526,103.0
9411,133,0,Белоруссия,Минская обл.,10942,5,2.604727,2.106269,1.302363,0.868242,0.464175,1.302363,0.000000,0.000000,0.000000,2.604727,2.604727,98.0
9412,133,0,Белоруссия,Могилевская обл.,10942,5,3.350220,2.106269,3.465078,3.366694,1.924089,3.465078,3.350220,3.260072,2.683042,3.579936,3.579936,145.0
9413,133,0,Белоруссия,г. Минск,10942,5,0.000000,2.106269,0.000000,0.000000,2.419962,0.000000,0.000000,0.000000,2.822072,0.000000,0.000000,0.0
9414,133,0,Казахстан,г. Нур-Султан,13301,5,2.678776,2.157494,1.339388,1.838330,1.741946,1.339388,2.678776,2.678776,2.577937,2.678776,2.836213,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12228,986,9943,Россия,Смоленская обл.,17460,7,0.000000,1.968393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
12229,998,0,Россия,Ленинградская обл.,18079,7,1.729716,1.968393,1.511099,1.535720,0.911504,1.511099,1.584963,1.438722,1.292481,1.729716,1.729716,5.0
12230,998,3380,Россия,Ленинградская обл.,14956,7,3.465369,1.968393,3.482684,3.416765,1.118352,3.482684,3.465369,3.375148,0.000000,3.500000,3.500000,121.0
12231,998,5410,Россия,г. Санкт-Петербург,14956,7,3.329106,1.968393,3.449441,3.452771,3.356213,3.449441,3.459432,3.514604,3.329106,3.569776,3.569776,120.0


TRAIN:  0 ... 12232
TEST:  12233 ... 15055
Обучающий набор: 13.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
12233,124,7278,Россия,Респ. Татарстан,17460,8,0.000000,2.008927,1.494320,2.112953,3.464547,1.494320,2.988640,3.169430,3.825526,2.988640,3.350220,0.0
12234,133,0,Белоруссия,Минская обл.,10942,8,2.988640,2.138527,3.088080,3.163613,1.007964,3.088080,3.187520,3.088080,0.000000,3.187520,3.314678,145.0
12235,133,0,Белоруссия,Могилевская обл.,10942,8,0.000000,2.138527,0.000000,1.198304,1.926184,0.000000,0.000000,1.675110,2.683042,0.000000,3.594912,0.0
12236,133,0,Белоруссия,г. Минск,10942,8,0.000000,2.138527,0.000000,0.000000,1.578783,0.000000,0.000000,0.000000,1.098079,0.000000,0.000000,41.0
12237,133,0,Казахстан,г. Нур-Султан,13301,8,2.678776,2.082592,2.678776,1.785851,1.965177,2.678776,2.678776,2.678776,2.678776,2.678776,2.678776,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15051,986,9943,Россия,Смоленская обл.,17460,10,0.000000,2.086452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
15052,998,0,Россия,Ленинградская обл.,18079,10,1.729716,2.086452,1.511099,1.438226,1.066607,1.511099,1.292481,1.511099,1.292481,1.729716,1.729716,10.0
15053,998,3380,Россия,Ленинградская обл.,14956,10,3.441322,2.086452,3.476253,3.472625,1.986508,3.476253,3.465369,3.465369,3.227426,3.511184,3.511184,115.0
15054,998,5410,Россия,г. Санкт-Петербург,14956,10,3.169925,2.086452,3.314678,3.362929,3.391199,3.314678,3.459432,3.394269,3.456438,3.459432,3.459432,120.0


TRAIN:  0 ... 15055
TEST:  15056 ... 17878
Обучающий набор: 16.000000
Тестовый набор: 3


...

TRAIN:  0 ... 17878
TEST:  17879 ... 20701
Обучающий набор: 19.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
17879,124,7278,Россия,Респ. Татарстан,17460,2,0.000000,2.146407,0.000000,0.000000,1.479958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
17880,133,0,Белоруссия,Минская обл.,10942,2,2.084963,2.440112,2.967591,3.262954,2.726344,2.967591,3.850220,3.802059,3.251099,3.850220,3.853680,72.0
17881,133,0,Белоруссия,Могилевская обл.,10942,2,0.000000,2.440112,0.000000,0.000000,1.141249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
17882,133,0,Белоруссия,г. Минск,10942,2,0.000000,2.440112,0.000000,0.904377,0.982747,0.000000,0.000000,1.356566,0.000000,0.000000,2.713132,0.0
17883,133,0,Казахстан,г. Нур-Султан,13301,2,0.000000,2.293841,1.380890,1.865998,1.825924,1.380890,2.761781,2.761781,2.678776,2.761781,2.836213,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20697,986,9943,Россия,Смоленская обл.,17460,4,3.000000,2.140374,2.614858,1.743239,0.435810,2.614858,2.229716,1.114858,0.000000,3.000000,3.000000,125.0
20698,998,0,Россия,Ленинградская обл.,18079,4,1.292481,2.140374,1.292481,1.292481,1.327723,1.292481,1.292481,1.292481,1.292481,1.292481,1.292481,0.0
20699,998,3380,Россия,Ленинградская обл.,14956,4,3.471257,2.140374,3.292833,3.230131,3.298687,3.292833,3.114409,3.109568,3.435156,3.471257,3.471257,100.0
20700,998,5410,Россия,г. Санкт-Петербург,14956,4,3.749923,2.140374,3.749923,3.609651,3.469530,3.749923,3.749923,3.539514,3.459432,3.749923,3.749923,100.0


TRAIN:  0 ... 20701
TEST:  20702 ... 23524
Обучающий набор: 22.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
20702,124,7278,Россия,Респ. Татарстан,17460,5,0.000000,2.132447,0.000000,0.000000,0.528238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20703,133,0,Белоруссия,Минская обл.,10942,5,4.312854,2.427139,4.149313,3.797846,3.458745,4.149313,3.985772,3.540342,3.538902,4.312854,4.312854,288.0
20704,133,0,Белоруссия,Могилевская обл.,10942,5,3.691852,2.427139,1.845926,1.230617,0.607230,1.845926,0.000000,0.000000,0.000000,3.691852,3.691852,62.0
20705,133,0,Белоруссия,г. Минск,10942,5,2.229716,2.427139,1.114858,0.743239,1.168557,1.114858,0.000000,0.000000,0.000000,2.229716,2.229716,0.0
20706,133,0,Казахстан,г. Нур-Султан,13301,5,0.000000,2.090911,1.339388,1.838330,1.825924,1.339388,2.678776,1.339388,2.678776,2.678776,2.836213,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23520,986,9943,Россия,Смоленская обл.,17460,7,3.204695,2.068485,3.204695,3.299344,1.260646,3.204695,3.204695,3.204695,0.000000,3.204695,3.488640,83.0
23521,998,0,Россия,Ленинградская обл.,18079,7,1.000000,2.068485,1.000000,0.666667,1.110460,1.000000,1.000000,1.000000,1.292481,1.000000,1.000000,9.0
23522,998,3380,Россия,Ленинградская обл.,14956,7,2.477098,2.068485,2.238549,2.602068,3.095013,2.238549,2.477098,2.903102,3.336178,2.477098,3.329106,50.0
23523,998,5410,Россия,г. Санкт-Петербург,14956,7,2.678776,2.068485,3.124276,3.192552,3.404476,3.124276,3.329106,3.449441,3.459432,3.569776,3.569776,0.0


tscv.get_n_splits()= 5


In [17]:
# создаем списки категориальных признаков
#cat_columns = data.select_dtypes(include='object').columns
cat_columns = CAT_COLS

# создаем список количественных признаков
#num_columns = train.select_dtypes(exclude='object').columns.tolist()
#num_columns =  [x for x in cv_features.columns.to_list() if x not in CAT_COLS + [TARGET]]
num_columns =  [x for x in FTS_COLS if x not in CAT_COLS + [TARGET]]
print(cat_columns)
print(num_columns)

['material_code', 'company_code', 'region', 'month']
['vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']


> # ********  Phase 1 *************

In [18]:
# обучаем модель, используя логарифмирование зависимой 
train = cv_features[FTS_COLS]
y_train = cv_features[TARGET].clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG))

In [19]:
# создаем конвейеры
cat_pipe = Pipeline([#('imputer', SimpleImputer(strategy='most_frequent')), 
                     ('enc',TargetEncoder()
                     )])
num_pipe = Pipeline([#('imputer', SimpleImputer(strategy='median')),
                     ('scal', None)]) 
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [20]:
if Phase_1:
    params2 = params   
    # задаем конвейер
    ml_pipe = Pipeline([('tr', transformer), 
                        ('ml', lgb.LGBMRegressor(**params) )])

In [21]:
if Phase_1:
    # задаем сетку гиперпараметров   
    param_grid = [{
        #'tr__cat__imputer__strategy': ['most_frequent',  'constant'],
        'tr__cat__enc': [TargetEncoder(),
                         CatBoostEncoder(),
                         #LabelEncoder(), 
                         OrdinalEncoder(),
                         #OneHotEncoder(sparse=False, handle_unknown='ignore'),
                         #LeaveOneOutEncoder()
                        ],
        #'tr__num__imputer__strategy': ['mean', 'median', 'constant'],
        'tr__num__scal': [None], # StandardScaler()],
        'ml__learning_rate': RANG_LEARNING_RATE,
        'ml__n_estimators' : N_ESTIMATORS_RANG
    }]

In [22]:
rmsle = lambda y_true, y_pred: \
            np.sqrt(mean_squared_log_error(
                        np.clip(y_true,0,None), 
                        np.clip(y_pred,0,None)
                    ))

In [23]:
%%time

if Phase_1:
    # создаем экземпляр класса RandomizedSearchCV или GridSearchCV, передав конвейер,
    # сетку гиперпараметров и указав количество
    # блоков перекрестной проверки, отключив запись метрик 
    # для обучающих блоков перекрестной проверки в атрибут cv_results_
    #gs = GridSearchCV(
    gs = RandomizedSearchCV( 
                      ml_pipe, 
                      param_grid, 
                      cv=tscv,  
                      scoring=make_scorer(rmsle, greater_is_better=False),
                      n_jobs=-1, 
                      return_train_score=False,
                      verbose=1,
                      random_state=SEED,
                    n_iter = RSCV_N_TRIALS
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
                     )
    # выполняем решетчатый поиск
    gs.fit(train, y_train)
    
    
    # записываем оптимальные значения гиперпараметров
    OPT_PARAMS_PIPE = gs.best_params_ 
    display(gs.best_params_, gs.best_score_)

Wall time: 0 ns


In [24]:
if Phase_1:
    #y_probas = gs.predict_proba(test)
    # смотрим наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(OPT_PARAMS_PIPE))
    # смотрим наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(gs.best_score_))
    # смотрим значение SCORE на тестовой выборке
    #print('Значение SCORE на тестовой выборке: {:.3f}'.format(
    #   roc_auc_score(y_test, y_probas[:, 1] )))

In [25]:
if Phase_1:
    # увеличиваем количество выводимых строк
    pd.set_option('display.max_rows', 300)

    # выводим результаты
    cv_results = pd.DataFrame(gs.cv_results_)[['mean_test_score',
                                               'param_ml__n_estimators',
                                               'param_ml__learning_rate',
                                               ]]
    cv_results = cv_results.sort_values(by='mean_test_score', ascending=False).reset_index(drop=True)
    for col in cv_results.columns[cv_results.columns.str.contains('enc')]:
        if col in 'mean_test_score':
            continue
        cv_results[col] = cv_results[col].apply(lambda x: type(x).__name__)
    display(cv_results)

> # ********  Phase 2,3,4 *************

In [26]:
if Phase_2 or Phase_3 or Phase_4:
    # задаем итоговый конвейер ПОКА ЧТО С НАЧАЛЬНЫМИ НАСТРОЙКАМИ
    ml_pipe2 = Pipeline([('tr', transformer), 
                         ('ml', lgb.LGBMRegressor(**params) )])

    # УСТАНОВКА В НОВЫЙ PIPE НАИЛУЧШИХ ЗНАЧЕНИЙ НАЙДЕННЫХ В ПРЕДЫДУЩЕЙ ФАЗЕ:  
    if Phase_1:  
        # присваиваем итоговому конвейеру оптимальные значения гиперпараметров
        print('Before modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators']) # 'n_estimators'
        print('Before modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate']) # 'n_estimators'
        ml_pipe2.set_params(**OPT_PARAMS_PIPE)
        print('After modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators'])
        print('After modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate'])

In [27]:
params2 = params3 = params4 = {}

if Phase_2: 
    params2 = {
        # (1) гиперпараметры, задающие структуру дерева:
       'ml__max_depth': IntUniformDistribution(1, 32), #2, 63 и бесконечное #6, 127),
        
       'ml__num_leaves': IntUniformDistribution(2, 1024),#  4095),# 131070), #4095),#  (10, 200)
       'ml__max_bin': IntUniformDistribution(1, 1024),  
       'ml__min_child_samples': IntUniformDistribution(1, 300),
       'ml__min_child_weight': UniformDistribution(0.01, 300),#(синонимы:min_child_weight,ml__min_sum_hessian_in_leaf...)
        }

In [28]:
 # (2) гиперпараметры, привносящие случайность в ходе построения дерева:
if Phase_3: 
    params3 = {
               'ml__bagging_fraction': UniformDistribution(0.0, 1.0),# синонимы:subsample,...
               'ml__colsample_bytree': UniformDistribution(0.0, 1.0), # (0.3, 0.9), #синонимы:colsample_bytree,feature_fraction,...
               'ml__subsample': IntUniformDistribution(1, 81, 5), # 'синонимы:subsample',bagging_freq,...
             }

In [29]:
# (3) гиперпараметры, задающие регуляризацию функции потерь:
if Phase_4: 
    params4 = {
                'ml__reg_alpha': UniformDistribution(1e-3, 25.0), #0,01 или 0,1
                'ml__reg_lambda': UniformDistribution(1e-3, 25.0), #0,01 или 0,1
             }

In [30]:
if Phase_2 or Phase_3 or Phase_4:  
    # задаем новое пространство поиска
    param_distributions = {**params2,**params3,**params4}
    print(param_distributions)

{'ml__reg_alpha': UniformDistribution(high=25.0, low=0.001), 'ml__reg_lambda': UniformDistribution(high=25.0, low=0.001)}


In [31]:
if Phase_2 or Phase_3 or Phase_4:  
    # создаем экземпляр класса OptunaSearchCV
    optuna_search = OptunaSearchCV(
        ml_pipe2,
        param_distributions,
        scoring=make_scorer(rmsle, greater_is_better=False),
        n_trials=N_TRIALS, # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#        n_trials=50,  # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        n_jobs=-1,
        refit=True,
        verbose=1,
        cv=tscv,   # CV_O,
        random_state=SEED)

C:\Users\rb102656\AppData\Local\Temp/ipykernel_3776/815814724.py:3: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(


In [32]:
%%time
if Phase_2 or Phase_3 or Phase_4:  
    # выполняем оптимизацию
    optuna_search.fit(train, y_train)

[I 2021-12-23 23:08:31,087] A new study created in memory with name: no-name-c997ad8b-6074-4d1e-88bc-659563481009
[I 2021-12-23 23:08:31,088] Searching the best hyperparameters using 23525 samples...
[I 2021-12-23 23:08:32,686] Trial 0 finished with value: -0.5303630196890897 and parameters: {'ml__reg_alpha': 4.123500805422892, 'ml__reg_lambda': 21.311610211766162}. Best is trial 0 with value: -0.5303630196890897.
[I 2021-12-23 23:08:32,743] Trial 1 finished with value: -0.5305578239194609 and parameters: {'ml__reg_alpha': 19.51642367216505, 'ml__reg_lambda': 16.10221538847366}. Best is trial 0 with value: -0.5303630196890897.
[I 2021-12-23 23:08:32,756] Trial 3 finished with value: -0.5305896749071511 and parameters: {'ml__reg_alpha': 21.823664519480175, 'ml__reg_lambda': 5.589726479576519}. Best is trial 0 with value: -0.5303630196890897.
[I 2021-12-23 23:08:32,760] Trial 2 finished with value: -0.5304726898183549 and parameters: {'ml__reg_alpha': 24.7769435825673, 'ml__reg_lambda': 

[I 2021-12-23 23:08:45,228] Trial 32 finished with value: -0.530474874035992 and parameters: {'ml__reg_alpha': 4.755900456987526, 'ml__reg_lambda': 5.4901848460946105}. Best is trial 21 with value: -0.5292503094882357.
[I 2021-12-23 23:08:46,547] Trial 36 finished with value: -0.5295732209133084 and parameters: {'ml__reg_alpha': 1.0318464330345538, 'ml__reg_lambda': 1.3409164929989033}. Best is trial 21 with value: -0.5292503094882357.
[I 2021-12-23 23:08:46,675] Trial 37 finished with value: -0.5303099777923018 and parameters: {'ml__reg_alpha': 8.077348237667927, 'ml__reg_lambda': 1.6294001796511974}. Best is trial 21 with value: -0.5292503094882357.
[I 2021-12-23 23:08:46,764] Trial 39 finished with value: -0.5298317240702209 and parameters: {'ml__reg_alpha': 0.020773669060393685, 'ml__reg_lambda': 2.0105590807897746}. Best is trial 21 with value: -0.5292503094882357.
[I 2021-12-23 23:08:46,826] Trial 38 finished with value: -0.5299479241691214 and parameters: {'ml__reg_alpha': 8.664

[LightGBM] [Warning] bagging_fraction is set=0.8276306589058549, subsample=21 will be ignored. Current value: bagging_fraction=0.8276306589058549
Wall time: 19.7 s


In [33]:
if Phase_2 or Phase_3 or Phase_4:  
    # записываем оптимальные значения гиперпараметров optuna
    opt_optuna_params_pipe = optuna_search.best_params_
    if Phase_1:
        opt_optuna_params_pipe['ml__learning_rate'] = OPT_PARAMS_PIPE['ml__learning_rate']
        opt_optuna_params_pipe['ml__n_estimators'] = OPT_PARAMS_PIPE['ml__n_estimators']

    # печатаем наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(opt_optuna_params_pipe))
    # печатаем наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(optuna_search.best_score_))
    # смотрим значение SCORE на тестовой выборке
    #print('SCORE на тестовой выборке: {:.3f}'.format(
    #    rsmle(y_test, optuna_search.predict_proba(test)[:, 1])))
    print('')

Наилучшие значения гиперпараметров: {'ml__reg_alpha': 1.0796993492912197, 'ml__reg_lambda': 0.062264967128417226}
Наилучшее значение SCORE: -0.529



#### Найденные на каждой фазе значения записываем в params = {...} в ячейке вверху ноутбука

> # ********  Final phase *************

In [34]:
# формируем полный массив меток и массив признаков

In [35]:
fulldata, y_fulldata = full_features[FTS_COLS]  , full_features[TARGET]

In [36]:
# трансформеры
final_cat_pipe = Pipeline([('enc',TargetEncoder() )])  #OneHotEncoder(handle_unknown='ignore', sparse=False))])
final_num_pipe = Pipeline([('scal', None)]) # 

final_transformers = [('num', final_num_pipe, num_columns),
                     ('cat', final_cat_pipe, cat_columns),]

# передаем список трансформеров в ColumnTransformer
final_transformer = ColumnTransformer(transformers=final_transformers)

# задаем итоговый конвейер ПОКА ЧТО С НАЧАЛЬНЫМИ НАСТРОЙКАМИ
final_ml_pipe = Pipeline([('tr', final_transformer), 
                     ('ml', lgb.LGBMRegressor(**params) )])


if Phase_1:
    final_ml_pipe.set_params(**OPT_PARAMS_PIPE)


if Phase_2 or Phase_3 or Phase_4:
    # Для контроля изменений смотрим выбранные параметры модели 
    opt_par = ['ml__n_estimators','ml__learning_rate',
               'ml__max_depth','ml__min_child_weight','ml__subsample']
    print(opt_par)
    print( 'Before modification ml_pipe3: ' ,[final_ml_pipe.get_params()[x] for x in opt_par])
    
    # УСТАНОВКА В НОВЫЙ PIPE НАИЛУЧШИХ ЗНАЧЕНИЙ НАЙДЕННЫХ В ПРЕДЫДУЩИХ ФАЗАХ:  
    # присваиваем итоговому конвейеру оптимальные значения гиперпараметров найденные optuna
    final_ml_pipe.set_params(**opt_optuna_params_pipe)
    
    # Для контроля изменений
    print( 'After modification ml_pipe3: ' ,[final_ml_pipe.get_params()[x] for x in opt_par])

['ml__n_estimators', 'ml__learning_rate', 'ml__max_depth', 'ml__min_child_weight', 'ml__subsample']
Before modification ml_pipe3:  [106, 0.055, 4, 137.32283812349354, 21]
After modification ml_pipe3:  [106, 0.055, 4, 137.32283812349354, 21]


In [37]:
# Нужно увеличить n_estimators, ЕСЛИ параметры были найдены на уменьшенном наборе данных
CV = 24 
par={}

par['ml__n_estimators'] = int(final_ml_pipe.get_params()['ml__n_estimators'] * (1 + 1/(CV-1)))
final_ml_pipe.set_params(**par )

par

{'ml__n_estimators': 110}

In [38]:
%%time

# обучаем обновленную модель на всей обучающей выборке
final_ml_pipe.fit(
                fulldata[FTS_COLS], 
                y_fulldata.clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG ))
                )

Wall time: 150 ms


Pipeline(steps=[('tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scal',
                                                                   None)]),
                                                  ['vol_tm1',
                                                   'country_grp_mean_10',
                                                   'avg_2', 'avg_3', 'avg_12',
                                                   'median2', 'median3',
                                                   'median4', 'median12',
                                                   'max_2', 'max_3']),
                                                 ('cat',
                                                  Pipeline(steps=[('enc',
                                                                   TargetEncoder())]),
                                                  ['material_code',
                                                  

In [39]:
dill.dump(final_ml_pipe, file = open("pipe_LGB_1.cbm", "wb"))

In [40]:
%%writefile predict.py

import pathlib
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import dill
import math
BASE_LOG = 4 # math.e

MODEL_FILE = pathlib.Path(__file__).parent.joinpath("pipe_LGB_1.cbm")

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
FTS_COLS = ['material_code', 'company_code', 'region', 'month', 'vol_tm1', 'country_grp_mean_10', 
             'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']
TARGET = "target"

# 
def get_features(df: pd.DataFrame, month: pd.Timestamp, N=1) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(N)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    # формируем лаги за N месяцев
    features[[f"vol_tm{i}" for i in range(N, 0, -1)]] = df.loc[:, start_period:end_period].copy()
 
    # Добавление ГРУППОВЫХ скользящих средних
    gr = "country"
    period = 10
    df2 = df.copy()
    df2[df2.columns.to_list()] = \
                           df2.groupby(level=gr).transform(lambda x: x.mean())
    grp_manager_roll_mean = df2.rolling(period, axis=1, min_periods=1)
    features = \
        features.join( grp_manager_roll_mean.mean().iloc[:, -1].rename(gr+"_grp_mean_"+str(period)))

    #  MEAN
    for period in [2,3,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.mean().iloc[:, -1].rename("avg_"+str(period)))

    #  median
    for period in [2,3,4,12]: # 10,11,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.median().iloc[:, -1].rename("median"+str(period)))
                 
    #  MAX
    for period in range(2,4,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.max().iloc[:, -1].rename("max_"+str(period)))

    features["month"] = month.month

    return features.reset_index()


def predict(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    model = dill.load(open(MODEL_FILE, "rb"))

    group_ts = df.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
    
    tmp = group_ts.clip(lower=1.0e-200).copy()
    for i in tmp.columns:
        tmp[i] = tmp[i].apply(lambda x: math.log(x+1, BASE_LOG))
    
    features = get_features(tmp, month)

    for c in features.columns:
        col_type = features[c].dtype
        if col_type == 'object': 
            features[c] = features[c].astype('category')    
    
    predictions = model.predict(features[FTS_COLS])
    predictions = BASE_LOG**predictions - 1
    
    predictions = pd.Series(predictions).clip(lower=0, upper=7000)

    preds_df = features[AGG_COLS].copy()
    preds_df["prediction"] = predictions
    return preds_df

Overwriting predict.py


In [41]:
import predict
import importlib
importlib.reload(predict)

<module 'predict' from 'E:\\Dev\\Jupyter\\dist\\SIBUR2021\\predict.py'>

In [42]:
ts_preds_tst = predict.predict(data[data.month<"2020-07-01"], pd.Timestamp("2020-07-01"))
ts_preds_tst 

,material_code,company_code,country,region,manager_code,prediction
0,124,7278,Россия,Респ. Татарстан,17460,0.884473
1,133,0,Белоруссия,Минская обл.,10942,122.087050
2,133,0,Белоруссия,Могилевская обл.,10942,4.295504
3,133,0,Белоруссия,г. Минск,10942,1.849671
4,133,0,Казахстан,г. Нур-Султан,13301,14.395172
...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,54.923496
937,998,0,Россия,Ленинградская обл.,18079,1.912615
938,998,3380,Россия,Ленинградская обл.,14956,31.009524
939,998,5410,Россия,г. Санкт-Петербург,14956,64.799522


In [43]:
print("Ошибка на тестовом множестве:",
      f'{np.sqrt(mean_squared_log_error(group_ts.reset_index().iloc[:,-1], ts_preds_tst["prediction"])):.4f}')

Ошибка на тестовом множестве: 1.5340


In [44]:
# Windows
!tar.exe -a -c -f Pip_LGB_NF_L4__8.zip pipe_LGB_1.cbm requirements.txt predict.py

In [45]:
end_time = time()
time_taken = end_time - start_time # time_taken is in seconds
hours, rest = divmod(time_taken,3600)
minutes, seconds = divmod(rest, 60)
print('Время работы скрипта:: {} hours {} minutes {} seconds.'.format(hours, minutes, seconds))

Время работы скрипта:: 0.0 hours 0.0 minutes 39.82309317588806 seconds.
